# Syntetic data-generator with Faker
### 1. Create data-tables

In [63]:
# IMPORTS
from datetime import datetime, timedelta
import random
from faker import Faker
import pandas as pd
import numpy as np

In [64]:
# Choose number of datasamples
num_samples = 1000

# Set filepath for saving
file_path_general = 'C:/Users/f_obersteiner/Desktop/Testdaten/final_data/'

In [65]:
# Generate valid data for Versand-data
current_time = datetime.now()
random_month = random.randint(1, 12)
one_year_ago = current_time.replace(year=current_time.year-1, month=random_month)
# print(one_year_ago)

# Initialisieren des Faker-Moduls
fake = Faker()

# Generieren von Daten für eine neue Zeile
def generate_valid_data():
    startzeitpunkt = one_year_ago
    dif_minutes = fake.random_int(min=2, max=240)
    endzeitpunkt = startzeitpunkt + timedelta(minutes=dif_minutes)
    # print("Endzeitpunkt: " + endzeitpunkt.strftime('%d.%m.%Y %H:%M'))
    dif_minutes1 = fake.random_int(min=2, max=240)
    packzeitpunkt = endzeitpunkt + timedelta(minutes=dif_minutes1)
    # print("Packzeitpunkt: " + packzeitpunkt.strftime('%d.%m.%Y %H:%M'))
    dif_minutes2 = fake.random_int(min=2, max=240)
    verladestart = packzeitpunkt + timedelta(minutes=dif_minutes2)
    # print("Verladestart: " + verladestart.strftime('%d.%m.%Y %H:%M'))
    dif_minutes3 = fake.random_int(min=2, max=240)
    verladeende = verladestart + timedelta(minutes=dif_minutes3)
    # print("Verladeende: " + verladeende.strftime('%d.%m.%Y %H:%M'))

    #auftragsnummer = fake.random_int(min=100, max=999)
    #auftragsnummer = fake.pyint(min_value=100, max_value=999)

    # Es wird zuerst der Mittelwert mu und die Standardabweichung sigma für die Normalverteilung definiert. 
    # Dann wird die Methode random_int() verwendet. 
    # Der min-Parameter wird auf eine zufällige Zahl gesetzt, die aus einer Normalverteilung mit dem Mittelwert mu und der Standardabweichung sigma stammt. 
    # Der step-Parameter wird zufällig aus den Werten 1, 2 oder 3 gewählt. Dadurch wird sichergestellt, dass einige der zufälligen Zahlen doppelt oder dreifach vorkommen können.
    
    # Mittelwert und Standardabweichung für die Normalverteilung
    mu = 50
    sigma = 10
    # Erstellen einer Auftragsnummer mit 4 Zeichen
    #auftragsnummer = fake.random_int(min=int(np.random.normal(mu, sigma)), max=9999, step=random.choice([1, 2, 3]))
    # Die generierte Zahl wird mit np.random.normal(mu, sigma) erzeugt und dann mit np.round() auf vier Stellen gerundet. 
    # Die gerundete Zahl wird dann mit der max()-Funktion verglichen, um sicherzustellen, dass sie mindestens vier Stellen hat. 
    # Wenn die gerundete Zahl kleiner als 1000 ist, wird sie auf 1000 gesetzt. 
    # Das Ergebnis ist eine normalverteilte Zufallszahl mit mindestens vier Stellen.
    auftragsnummer = fake.random_int(min=max(int(np.round(np.random.normal(mu, sigma))), 1000), max=9999, step=random.choice([1, 2, 3]))

    versand_id = fake.random_int(min=10, max=99)
    verladestart_str = verladestart.strftime('%d.%m.%Y %H:%M')
    verladeende_str = verladeende.strftime('%d.%m.%Y %H:%M')
    versand_soll_str = (
        verladestart + timedelta(minutes=30)).strftime('%d.%m.%Y %H:%M')

    # Erstellen der neuen Datenreihe als Liste
    new_row = [auftragsnummer, versand_id, verladestart_str, verladeende_str, versand_soll_str]

    return new_row

In [66]:
%%capture
# Create  Versand-data-file as csv
df_VER = pd.DataFrame(columns=['Auftragsnummer', 'Versand_ID', 'Verladestart', 'Verladeende', 'Versand_SOLL'])
    # Schleife über die Anzahl der Datensätze
for i in range(num_samples):
    # Schreiben der neuen Zeile in die CSV-Datei
    new_row = generate_valid_data()
    # print(new_row)
    df_VER = df_VER.append(pd.Series(new_row, index=df_VER.columns), ignore_index=True)
    
file_path = file_path_general + "syntetic_VER_data.csv"
df_VER.to_csv(file_path, index=False)


In [67]:
# Generate valid data for Kommissionier-data

# Initialisieren des Faker-Moduls
fake = Faker()

# Generieren von Daten für eine neue Zeile


def generate_valid_data2(verladestart_ue, auftragsnummer_ue):
  
    verladestart_to_date = datetime.strptime(verladestart_ue, '%d.%m.%Y %H:%M')
    dif_minutes1 = fake.random_int(min=2, max=240)
    packzeitpunkt = verladestart_to_date - timedelta(minutes=dif_minutes1)

    auftragsnummer = auftragsnummer_ue
    lagereinheit_id = fake.random_int(min=100, max=999)
    packplatz = "Packplatz " + str(random.randint(1, 10)).zfill(2)
    packzeitpunkt_str = packzeitpunkt.strftime('%d.%m.%Y %H:%M')
    menge = fake.random_int(min=1, max=500)

    # Erstellen der neuen Datenreihe als Liste
    new_row = [lagereinheit_id, packplatz, auftragsnummer, packzeitpunkt_str, menge]

    return new_row

In [68]:
%%capture
# Create  KOM-data-file as csv
df_KOM = pd.DataFrame(columns=['Lagereinheit_ID', 'Packplatz', 'Auftragsnummer', 'Packzeitpunkt', 'Menge'])

    # Schleife über die Anzahl der Datensätze
for i in range(num_samples):
    # Schreiben der neuen Zeile in die CSV-Datei
    new_row = generate_valid_data2(df_VER.iloc[i, 2], df_VER.iloc[i, 0])
    # print(new_row)
    df_KOM = df_KOM.append(pd.Series(new_row, index=df_KOM.columns), ignore_index=True)

file_path = file_path_general + "syntetic_KOM_data.csv"
df_KOM.to_csv(file_path, index=False)

print(df_KOM)

In [69]:
# Generate valid data for WMS-data

# Initialisieren des Faker-Moduls
fake = Faker()

# Generieren von Daten für eine neue Zeile


def generate_valid_data3(lagereinheit_id_ue, packzeitpunkt_ue, lagerort_von_ue, startzeitpunkt_ue):
    
    packzeitpunkt_to_date = datetime.strptime(packzeitpunkt_ue, '%d.%m.%Y %H:%M')
    startzeitpunkt_to_date = datetime.strptime(startzeitpunkt_ue, '%d.%m.%Y %H:%M')
    

    
    lagereinheit_id = lagereinheit_id_ue

    if lagerort_von_ue == 'Wareneingang':
     lagerort_von = 'Wareneingang'
     lagerort_nach = 'Hochregallager'
     dif_minutes1 = fake.random_int(min=2, max=240)
     endzeitpunkt = startzeitpunkt_to_date - timedelta(minutes=dif_minutes1)
     dif_minutes2 = fake.random_int(min=2, max=240)
     startzeitpunkt = endzeitpunkt - timedelta(minutes=dif_minutes2)
    else: 
     lagerort_von = 'Hochregallager'
     lagerort_nach = 'Packplatz'
     dif_minutes1 = fake.random_int(min=2, max=240)
     endzeitpunkt = packzeitpunkt_to_date - timedelta(minutes=dif_minutes1)
     dif_minutes2 = fake.random_int(min=2, max=240)
     startzeitpunkt = endzeitpunkt - timedelta(minutes=dif_minutes2)

    # Erstellen der neuen Datenreihe als Liste
    new_row = [lagereinheit_id, lagerort_von, lagerort_nach, startzeitpunkt, endzeitpunkt]

    return new_row

In [70]:
%%capture
# Create  WMS-data-file as csv
df_WMS = pd.DataFrame(columns=['Lagereinheit_ID', 'Lagerort_von', 'Lagerort_nach', 'Startzeitpunkt', 'Endzeitpunkt'])

# Schleife über die Anzahl der Datensätze
a = 0
while a < num_samples:
   # print(a)
    lagereinheit_von_ue = 'Hochregallager'
    startzeitpunkt_ue = df_KOM.iloc[a, 3]
    #print(startzeitpunkt_ue)
   
    for i in range(2):
        #print(i)
        # Schreiben der neuen Zeile in die CSV-Datei
        new_row = generate_valid_data3(df_KOM.iloc[a, 0], df_KOM.iloc[a, 3], lagereinheit_von_ue, startzeitpunkt_ue)
        # print(new_row)
        df_WMS = df_WMS.append(pd.Series(new_row, index=df_WMS.columns), ignore_index=True)

        lagereinheit_von_ue = 'Wareneingang'
        startzeitpunkt_ue = df_WMS.iloc[a, 3].strftime('%d.%m.%Y %H:%M')
        #endzeitpunkt_ue = df_WMS.iloc[a, 4]

    a = a + 1
       
        #print(i)
        
#delete duplicates of Lagereinheit_ID
#Überprüfen welche Zeilen doppelte Werte in der Spalte haben
duplicates = df_WMS.duplicated(subset='Lagereinheit_ID')

#Löschen der doppelten Zeilen aus dem Dataframe
df_WMS_unique = df_WMS.drop_duplicates(subset=['Lagereinheit_ID', 'Lagerort_von'])

file_path = file_path_general + "syntetic_WMS_data.csv"
df_WMS_unique.to_csv(file_path, index=False)

#print(df_WMS)

### 2. Merge datatables

In [71]:
# IMPORTS
import numpy as np
import pandas as pd 

In [72]:
# data import
data_WMS = pd.read_csv('C:/Users/f_obersteiner/Desktop/Testdaten/final_data/syntetic_WMS_data.csv', delimiter=',')
data_KOM = pd.read_csv('C:/Users/f_obersteiner/Desktop/Testdaten/final_data/syntetic_KOM_data.csv', delimiter=',')
data_VER = pd.read_csv('C:/Users/f_obersteiner/Desktop/Testdaten/final_data/syntetic_VER_data.csv', delimiter=',')

In [73]:
# Merge
data_VER_KOM = data_VER.merge(data_KOM, left_on='Auftragsnummer', right_on='Auftragsnummer')
# print(data_VER_KOM)
data_VER_KOM_WMS = data_VER_KOM.merge(data_WMS, left_on='Lagereinheit_ID', right_on='Lagereinheit_ID')

# 'Datum'-Spalte in ein datetime-Objekt konvertieren
data_VER_KOM_WMS['Startzeitpunkt'] = pd.to_datetime(data_VER_KOM_WMS['Startzeitpunkt'])

# sort by 'Auftragsnummer' und 'Startzeitpunkt' in ascending order
data_VER_KOM_WMS.sort_values(by=['Auftragsnummer', 'Startzeitpunkt'], inplace=True)
# print(data_VER_KOM_WMS['Auftragsnummer'])

# create csv
file_path = file_path_general + "syntetic_merged_data.csv"
data_VER_KOM_WMS.to_csv(file_path, sep=",", index=False)